In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
%matplotlib inline

In [26]:
msft = yf.Ticker("MSFT")
bs = msft.balance_sheet
cf = msft.cashflow
income = msft.financials

In [39]:
msft.info['marketCap']

1256063107072

In [40]:
mu = yf.Ticker("MU")

In [42]:
mu.info['marketCap']

51305324544